In [3]:
!python -m pip install mediapipe


In [4]:
import os
import pandas
import cv2
import torch
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
cap=cv2.VideoCapture(r"C:\Users\ronak\OneDrive\Desktop\bbaf2n.mpg")
if not cap.isOpened():
    print("Error: Could not open video.")

In [ ]:
frames=[]
res=[]
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
    res.append(ret)

In [ ]:
rgb = cv2.cvtColor(frames[0], cv2.COLOR_BGR2RGB)

plt.imshow(rgb)
plt.axis('off')  # hide axes
plt.show()


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

base_options = python.BaseOptions(
    model_asset_path=r"C:\Users\ronak\Downloads\face_landmarker.task"
)


options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    num_faces=1
)


detector = vision.FaceLandmarker.create_from_options(options)




In [ ]:
mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=rgb
    )

result = detector.detect(mp_image)

In [ ]:
result.face_landmarks[0][0].x

In [ ]:
# MediaPipe lip landmark indices
LIPS = [
    61, 146, 91, 181, 84, 17, 314, 405,
    321, 375, 291, 308, 324, 318, 402,
    317, 14, 87, 178, 88, 95, 185,
    40, 39, 37, 0, 267, 269, 270,
    409, 415, 310, 311, 312, 13,
    82, 81, 42, 183, 78
]


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ---------- Load FaceLandmarker ----------
base_options = python.BaseOptions(
    model_asset_path=r"C:\Users\ronak\Downloads\face_landmarker.task"
)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    num_faces=1
)

detector = vision.FaceLandmarker.create_from_options(options)


cap = cv2.VideoCapture(r"C:\Users\ronak\OneDrive\Desktop\bbaf2n.mpg")

mouth_frames = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=rgb
    )

    result = detector.detect(mp_image)

    if not result.face_landmarks:
        continue

    landmarks = result.face_landmarks[0]

    xs = [int(landmarks[i].x * w) for i in LIPS]
    ys = [int(landmarks[i].y * h) for i in LIPS]

    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)

    mouth = frame[y_min:y_max, x_min:x_max]

    # Grayscale + resize (LipNet standard)
    mouth = cv2.cvtColor(mouth, cv2.COLOR_BGR2GRAY)
    mouth = cv2.resize(mouth, (100, 50))

    mouth_frames.append(mouth)

cap.release()

print("Total mouth frames:", len(mouth_frames))


In [ ]:
plt.imshow(mouth, cmap="gray")
plt.axis("off")
plt.show()


In [5]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

class Frames():
    def __init__(self,model_path):
        base_options = python.BaseOptions(
            model_asset_path=model_path)
        options = vision.FaceLandmarkerOptions(
            base_options=base_options,
            num_faces=1
        )
        self.detector = vision.FaceLandmarker.create_from_options(options)


        # MediaPipe lip landmark indices
        self.LIPS = [
            61, 146, 91, 181, 84, 17, 314, 405,
            321, 375, 291, 308, 324, 318,
            402, 317, 14, 87, 178, 88
        ]
        self.FEAT_DIM = len(self.LIPS) * 3 

    def extract_mouth_frames(self,video_path):
        cap = cv2.VideoCapture(video_path)

        frame_features = []
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
        
            h, w, _ = frame.shape
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
            mp_image = mp.Image(
                image_format=mp.ImageFormat.SRGB,
                data=rgb
            )
        
            result = self.detector.detect(mp_image)
        
            if not result.face_landmarks:
                continue
        
            face_landmarks = result.face_landmarks[0]

            landmarks = []
            for idx in self.LIPS:
                lm = face_landmarks[idx]
                landmarks.extend([lm.x, lm.y, lm.z])

            frame_feat = torch.tensor(landmarks)

            if frame_feat.numel() == self.FEAT_DIM:
                frame_features.append(frame_feat)
            # xs = [int(landmarks[i].x * w) for i in self.LIPS]
            # ys = [int(landmarks[i].y * h) for i in self.LIPS]
        
            # x_min, x_max = min(xs), max(xs)
            # y_min, y_max = min(ys), max(ys)
        
            # mouth = frame[y_min:y_max, x_min:x_max]
        
            
            # mouth = cv2.cvtColor(mouth, cv2.COLOR_BGR2GRAY)
            # mouth = cv2.resize(mouth, (100, 50))
        
            # mouth_frames.append(mouth)
        
        cap.release()
        

        if len(frame_features) == 0:
            return None

        # Shape: [T, 60]
        return torch.stack(frame_features)

ModuleNotFoundError: No module named 'mediapipe'

In [6]:
import sys
print(sys.executable)


C:\Users\ronak\anaconda3\python.exe


In [ ]:
frames=Frames("C:\Users\ronak\Downloads\face_landmarker.task")
a=frames.extract_mouth_frames("C:\Users\ronak\OneDrive\Desktop\bbaf2n.mpg")
plt.imshow(a, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
base_options = python.BaseOptions(
    model_asset_path=r"C:\Users\ronak\Downloads\face_landmarker.task"
)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    num_faces=1
)

detector = vision.FaceLandmarker.create_from_options(options)


# MediaPipe lip landmark indices
LIPS = [
    61, 146, 91, 181, 84, 17, 314, 405,
    321, 375, 291, 308, 324, 318, 402,
    317, 14, 87, 178, 88, 95, 185,
    40, 39, 37, 0, 267, 269, 270,
    409, 415, 310, 311, 312, 13,
    82, 81, 42, 183, 78
]


def extract_mouth_frames(video_path):
    cap = cv2.VideoCapture(video_path)

    mouth_frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
        mp_image = mp.Image(
            image_format=mp.ImageFormat.SRGB,
            data=rgb
        )
    
        result = detector.detect(mp_image)
    
        if not result.face_landmarks:
            continue
    
        landmarks = result.face_landmarks[0]
    
        xs = [int(landmarks[i].x * w) for i in LIPS]
        ys = [int(landmarks[i].y * h) for i in LIPS]
    
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)
    
        mouth = frame[y_min:y_max, x_min:x_max]
    
        
        mouth = cv2.cvtColor(mouth, cv2.COLOR_BGR2GRAY)
        mouth = cv2.resize(mouth, (100, 50))
    
        mouth_frames.append(mouth)
    
    cap.release()
    

    return mouth_frames


In [ ]:
def decode_align(path):
    words=[]
    with open(path,'r') as fobj:
        for line in fobj:
            _,_,word=line.strip().split()
            if word!= "sil":
                words.append(word)
    return " ".join(words)

In [ ]:
a=decode_align(r"C:\Users\ronak\OneDrive\Desktop\bbaf2n.align")
a

In [ ]:
vocab = list("abcdefghijklmnopqrstuvwxyz1234567890!ABCDEFGHIJKLMNOPQRSTUVWXYZ ")
char_to_idx = {c: i+1 for i, c in enumerate(vocab)}  # 0 reserved for CTC blank
idx_to_char = {i: c for c, i in char_to_idx.items()}


In [ ]:
def text_to_labels(text):
    output=[]
    for c in text:
        output.append(char_to_idx[c])
    return output


In [ ]:
text_to_labels(a)

In [ ]:
path=r"C:\Users\ronak\OneDrive\Desktop\data"
alignment_path=os.path.join(path,"alignments","s1")
speaker_path=os.path.join(path,"s1")
iterable=zip(os.listdir(alignment_path),os.listdir(speaker_path))
def load_data(path,iterable, batch_size=16):
    all_align = []
    all_video_frames = []
    alignment_path=os.path.join(path,"alignments","s1")
    speaker_path=os.path.join(path,"s1")
    for _ in range(batch_size):
        try:
            align_file, video_file = next(iterable)

            text=decode_align(os.path.join(alignment_path,align_file))
            decode_text=text_to_labels(text)

            video_file_path=os.path.join(alignment_path,align_file).split("\\")[-1][0:-5]+"mpg"
            
            frames=extract_mouth_frames(os.path.join(speaker_path,video_file_path))
            
            
            all_align.append(decode_text)
            all_video_frames.append(frames)
        except StopIteration:
            break

    return all_align, all_video_frames


In [ ]:
path=r"C:\Users\ronak\OneDrive\Desktop\data"
alignment_path=os.path.join(path,"alignments","s1")
speaker_path=os.path.join(path,"s1")
iterable=zip(os.listdir(alignment_path),os.listdir(speaker_path))
a,b=load_data(path,iterable,batch_size=12)

In [ ]:
len(b)

In [ ]:
p=r"C:\Users\ronak\OneDrive\Desktop\data\alignments\s1\bbal7s.align"
p.split("\\")[-1][0:-5]+"mpg"